# Proof of concept validation notebook for LEPR upload

In [1]:
import pandas as pd
import numpy as np
import logging
from pylepr import validate

In [2]:
upload_data = pd.read_excel('../data/upload_validation.xlsx', sheet_name=None)

In [5]:
validate.validate_upload(upload_data)

---> ERROR_(validate_chem_error_columns):: 'TiO2_err' missing from chemistry data columns.
---> ERROR_(validate_chem_error_columns):: 'FeO*_err' missing from chemistry data columns.
---> ERROR_(validate_chem_error_columns):: 'FeOtot_err' missing from chemistry data columns.
---> ERROR_(validate_chem_error_columns):: 'Al2O3_err' missing from chemistry data columns.
---> ERROR_(validate_chem_error_columns):: 'MgO_err' missing from chemistry data columns.
---> CRITICAL_(validate_chem_units):: 'Al2O3' does not provide any units.
---> CRITICAL_(validate_chem_method):: 'MgO' does not provide any method id.
---> ERROR_(numeric_chem_data_not_valid):: '(1)', the 'SiO2_err' value for exp_run 'Run_1', is not a valid number.
---> ERROR_(chem_measurement_limit_not_valid):: '<0.7', the 'TiO2' value for exp_run 'Run_4', is not valid. Instead give just the value and indicate limit using field '????, Ask roger'.
---> ERROR_(chem_not_detected_not_valid):: 'nd', the 'TiO2' value for exp_run 'Run_5', is n